# References

* [CS231n: Convolutional Neural Networks for Visual Recognition 2017](http://cs231n.stanford.edu/2017/syllabus)
    - [cs231n 2017 assignment #1 kNN, SVM, SoftMax, two-layer network](https://cs231n.github.io/assignments2017/assignment1/)
    - [Training a Softmax Linear Classifier](https://cs231n.github.io/neural-networks-case-study)
* [ゼロから作る Deep Learning](https://github.com/oreilly-japan/deep-learning-from-scratch)
* [Mathematics for Machine Learning](https://mml-book.github.io/)

---

# Neural network overview

Structure of the network and how forward and backward propagations work.

<img src="image/nn_diagram.png" align="left">

# Concepts 

## Objective function

The network trains the layers so as to minimize the objective function ```L``` which calculates the loss. Each layer at ```i``` is a function $f_i$ which takes an input $X_i$ from a previous layer and outputs $Y_i = f(X_i)$. The post layers of the form an objective function $L_i$ for the layer: $L = L_i(Y_i)$. 


<img src="image/nn_functions.png" align="left">

## Forward path

The process where each layer ```i``` calculate its output $Y_i = f(X_i)$ and forward it to the next layer(s) as their input $X_{i+1}$.

## Backward path

The process of automatic differentication, or *back-propagation* where each layer calculates its gradient $\frac {\partial L_i(Y_i)}{\partial Y_i}$ , that is, the impact $Y_i$ will make on the objective ```L``` when it changes. With the gradient, we can apply the gradient descent $X_i = X_i - \lambda  \frac {\partial L_i(Y_i)}{\partial Y_i} \frac {\partial Y_i }{\partial X_i}$ to update $X_i$ that would reduce the objective ```L```.

## Cycle

A round-trip of a forward path and a backward path with a batch data set $(X, T)$. How many cycles to happen with each batch is an implementation decision. 

## Epoch

Total cycles to consume the entire training data.

---

# Terminologies

## X
A batch input to a layer. Matrix shape is ```(N, D)```.

* ```N``` : Number of rows in a batch X, or batch size
* ```D``` : Number of features in a data in X.


## T
Labels for X. There are two formats available for the label.

#### One Hot Encoding (OHE) labels

When a neural network predicts a class out of ```3``` classes for an input ```x``` and the correct class is ```2```, then the label ```t``` is specified as ```t = [0, 1, 0]```.

$
\begin{align*}
\overset{ (N,M) }{ T_{_{OHE}} } &= ( 
    \overset{ (M,) }{ T_{(n=0)} }, \dots , \overset{ (M,) }{ T_{(n=N-1)} } 
) 
\\
\overset{ (M,) }{ T_{ _{OHE} (n)} } &= ( \overset{ () }{ t_{(n)(m=0)} }, \; \dots \;, \overset{ () }{ t_{(n)(m=M-1)} })
\end{align*}
$

#### Index labels

The label ```t``` is specified as ```t = 2```. 

$
\begin{align*}
\overset{ (N,) }{ T_{_{IDX}} } &= (\overset{ () }{ t_{(n=0)} }, \; \dots \;, \overset{ () }{ t_{(n=N-1)} }) \end{align*}
$

## W
A set of weight parameters of a node in a Matmul layer. Shape is ```(M, D)```.

* ```M``` : Number of nodes in a layer



# Matrix order

Use the row-order matrix. For instance, the weight matrix ```W``` of a Matmul layer has a shape ```(M, D)``` where each row in ```W``` represents a node in the layer. It will be efficient to use the column order matrix of shape ```(D, M)``` for ```W``` so that the matrix multiplication at a Matmul layer can be executed as ```X@W```  which is a ```shape:(N,D) @ shape:(D,M)``` operation without transpose. 

However, for the purpose of consistency and clarity, use the shape ```W:(M, D)``` although it will cause transposes ```W.T``` at the Matmul operations, and revese transposing ```dL/dW.T``` to ```dL/dW``` when updating ```W``` at the gradient descents.


---

# Python & Jupyter setups

In [ ]:
from typing import (
    Optional,
    Union,
    List,
    Dict,
    Tuple,
    Callable
)

### Python path
Python path setup to avoid the relative imports.

In [ ]:
import sys
import os 

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Package dependencies

In [ ]:
import inspect
from functools import partial
import copy
import logging
import numpy as np
import matplotlib.pyplot as plt

### Jupyter notebook

In [ ]:
#!conda install line_profile memory_profiler
%load_ext memory_profiler
%load_ext line_profiler

# Logging is enabled by calling logging.basicConfig
# logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
# Logger = logging.getLogger("neural_network")

%load_ext autoreload
%autoreload 2

### Matplotlib

In [ ]:
import matplotlib.style as mplstyle
mplstyle.use('fast')
plt.ion()

# Note: with notebook backend from the top, updating the plot line does not work...
%matplotlib notebook
# %matplotlib inline

### numpy

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=80) 

---

# Categorical Classification on Non-linearly separable data

Use Matmul and CrossEntropyLogLoss layers to classify M categorical data.

In [ ]:
%reload_ext autoreload
%autoreload 2

import common.weights as weights 
from common.function import (
    transform_X_T,
    softmax_cross_entropy_log_loss
)
from optimizer import (
    Optimizer,
    SGD
)
from network.test_040_two_layer_classifier import (
    train_two_layer_classifier
)
from common.function import (
    prediction_grid_2d
)

from drawing import (
    plot,
    scatter,
    COLOR_LABELS,   # labels to classify outside/0/red or inside/1/green.
    plot_categorical_predictions
)

## Data X and Label T
Training data set that cannot be linearly classified. ```X = ((A not B), (B not C), (C not A), (A and B and C and D))``` for circles A, B, C.

In [ ]:
%autoreload 2
from data import (
    sets_of_circle_A_not_B
)

In [ ]:
# Nuber of pots per class
__N = 500

# Number of classes
__M = 3        # Number of circles (A, B, C) and same with (A not B), (B not C), (C not A)
M = __M + 1    # Intersection of circies (A and B and C and D)

# Traing data (A not B), (B not C), (C not A) to classify
radius = 1
circles, centres, intersection = sets_of_circle_A_not_B(radius=radius, ratio=1.0, m=__M, n=__N)

# Stack all circles and intersect
X = np.vstack(
    [circles[i] for i in range(M-1)] + 
    [intersection]
)

T = np.hstack(
    [np.full(circles[i].shape[0], i) for i in range(M-1)] + 
    [np.full(intersection.shape[0], M-1)]
)
N = T.shape[0]
assert T.shape[0] == X.shape[0]

# Shuffle the data
indices = np.random.permutation(range(T.shape[0]))
X = X[indices]
T = T[indices]
Y = COLOR_LABELS[T]
X, T = transform_X_T(X, T)

x_min, x_max = X[:, 0].min(), X[:, 0].max()
y_min, y_max = X[:, 1].min(), X[:, 1].max()

print(f"X:{X.shape} T:{T.shape} ")

### Plot the data

In [ ]:
fig, ax = plt.subplots(figsize=(5,4)) 
ax.set_xlabel('x label')
ax.set_ylabel('y label')
ax.axis('equal')
ax.grid()
r = np.linspace(0, 2*np.pi, 100)

# (A not B), (B not C), (C not A)
for i in range(__M):
    circle = circles[i]
    if circle.size > 0:
        x = centres[i][0]
        y = centres[i][1]
        ax.scatter(circle[::, 0], circle[::, 1], color=COLOR_LABELS[i])
        ax.plot(
            x + radius * np.cos(r), 
            y + radius * np.sin(r), 
            linestyle='dashed', 
            color=COLOR_LABELS[i]
        )

# (A and B and C and D)
ax.scatter(intersection[::, 0], intersection[::, 1], color='gold')
plt.draw()
plt.show()
import time
time.sleep(1)
time.sleep(1)


## Training on non-linear separable data

During the training, the loss often does not decrease. 

> Iteration [19976]: Loss[0.06914290965513335] has not improved from the previous [0.06914225566912098] for 1 times.

<ins>If reduce the **learning rate** at those points, the situation gets worse </ins>(continuous non-improvements instead of sporadic) and the training fails (the result model cannot classify). If keep using the same learning rate, the non-improvement continues more frequently but the training itself makes a progress. 

Need to understand why it happens and why reducing the rate will make the training fail. Possibl approach is visualizing the loss function with contour lines and the track of the gradient descent to see the terrain it went through. 

### Network geometry

In [ ]:
D = 2
M1 = 8
M2: int = M                 # Nuber of output = Number of classes to classify

### Weight initialization
#### Trick
Because the data is almost zero-centered, the bias ```x0``` is not required. Hence set the bias weight ```w0``` to zero to short-cut the training. Without, the 

In [ ]:
W1 = weights.he(M1, D+1)
W2 = weights.he(M2, M1+1)

W1_bias_0 = copy.deepcopy(W1)  # np.copy() is sufficient without deepcopy.
W2_bias_0 = copy.deepcopy(W2)
W1_bias_0[
    ::,
    0
] = 0
W2_bias_0[
    ::,
    0
] = 0

### Gradient descent optimizer

In [ ]:
optimizer = SGD(lr=0.1, l2=1e-2)

### Training

In [ ]:
MAX_TEST_TIMES = 50000


W1_result_with_trick, W2_result_with_trick, objective, prediction_with_trick, history_with_trick = \
train_two_layer_classifier(
    N=N,
    D=D,
    X=X,
    T=T,
    M1=M1,
    W1=W1_bias_0,
    M2=M2,
    W2=W2_bias_0,
    log_loss_function=softmax_cross_entropy_log_loss,
    optimizer=optimizer,
    num_epochs=MAX_TEST_TIMES,
    test_numerical_gradient=False
)

### Plot predictions

In [ ]:
fig, ax = plt.subplots(figsize=(6,5)) 
x_grid, y_grid, predictions = prediction_grid_2d(x_min, x_max, y_min, y_max, prediction_with_trick)
plot_categorical_predictions(ax, [x_grid, y_grid], X, Y, predictions)

ax.set_xlabel('x label')
ax.set_ylabel('y label')
ax.axis('equal')
ax.grid()

### Plot training error

In [ ]:
_x = range(len(history_with_trick))
_y = history_with_trick
xlabel = 'iterations (log scale)'
ylabel = 'loss'
title = "training error"
fig, ax = plot(_x, _y, title=title, xlabel=xlabel, ylabel=ylabel,figsize=(5,4))
ax.set_ylim(0.0, 1.5)
ax.set_xscale('log')

### Training with weight without trick

In [ ]:
# W1 = np.copyto(W1, W1_backup)  # None will be set. Why not return the reference!?
W1_bias_not_0 = copy.deepcopy(W1)
W2_bias_not_0 = copy.deepcopy(W2)

W1_result_without_trick, W2_result_without_trick, objective, prediction_without_trick, history_without_trick=\
train_two_layer_classifier(
    N=N,
    D=D,
    X=X,
    T=T,
    M1=M1,
    W1=W1_bias_not_0,
    M2=M2,
    W2=W2_bias_not_0,
    log_loss_function=softmax_cross_entropy_log_loss,
    optimizer=optimizer,
    num_epochs=MAX_TEST_TIMES,
    test_numerical_gradient=False
)

### Prediction

In [ ]:
x_grid, y_grid, predictions = prediction_grid_2d(x_min, x_max, y_min, y_max, prediction_without_trick)

### Plot predictions

In [ ]:
fig, ax = plt.subplots(figsize=(6,5)) 
ax.set_xlabel('x label')
ax.set_ylabel('y label')
ax.axis('equal')
ax.grid()
plot_categorical_predictions(ax, [x_grid, y_grid], X, Y, predictions)

### Plot training error

In [ ]:
_x = range(len(history_without_trick))
_y = history_without_trick
xlabel = 'iterations (log scale)'
ylabel = 'loss'
title = "training error without trick"

fig, ax = plot(_x, _y, title=title, xlabel=xlabel, ylabel=ylabel,figsize=(5,4))
ax.set_ylim(0.0, 1.5)
ax.set_xscale('log')

## Batch normalization
Observe the effect of the batch normalization by inserting the layer in-between activation and matmul layers.